# Einstieg Selenium

In dieser Stunde schauen wir uns zum Einstieg wieder die Seite mit den Zitaten an. Allerdings widmen wir uns der Version der Seite https://quotes.toscrape.com/js/. Es handelt sich dabei um eine Version der Seite, bei der die Zitate mithilfe von JavaScript dargestellt werden. Zum Einstieg führen wir ein kleines Experiment durch. Öffnet euer Jupyter Notebook aus dem Abschnitt "Einstieg BeautifulSoup". Ersetzt URL = "https://quotes.toscrape.com" durch URL = "https://quotes.toscrape.com/js" und führt alle Codezellen aus. Was passiert?

Woran liegt das? Die Inhalte der Seite werden offensichtlich nicht vorgeladen, sondern sie werden erst beim Seitenaufruf ergänzt. Das könnt ihr nachvollziehen, indem ihr die Seite im Browser aufruft, die Entwicklertools öffnet, JavaScript deaktiviert, und die Seite https://quotes.toscrape.com/js/ neu ladet. Welche Inhalte der Seite werden beim neu Laden angezeigt? Wie ist das im Vergleich mit der Seite https://quotes.toscrape.com/?

### Was ist Selenium?

Zum Scrapen solcher Inhalte können wir BeautifulSoup nicht verwenden, weil BeautifulSoup mit solchen mithilfe von JavaScript dargestellten Inhalten nicht umgehen kann: In unserem quotes.toscrape-Webscraper haben wir ja nur eine HTTP-Anfrage gestellt, und diese Anfrage liefert nur die HTML-Dokumente, die bereits auf dem Server liegen, bevor wir die Webseite aufrufen. Wir müssen also irgendwie an die Inhalte kommen, welche mithilfe von JavaScript im Client nachträglich eingefügt werden.

Dazu gibt es verschiedene spezialisierte Python-Pakete und Bibliotheken, am gängigsten sind Selenium, die leichtgewichtigere Selenium-Variante Helium sowie Puppeteer und Playwright. Daneben gibt es auch Splash, das in Kombination mit Scrapy, eine BeautifulSoup-Alternative, verwendet wird. Das wohl meist verwendete Paket zum Scrapen dynamischer Inhalte war im letzten Jahrzehnt immer Selenium, allerdings wird es vermehrt durch Playwright verdrängt. Zum Verständnis von Playwright müssten wir uns aber noch ein anderes Netzwerkprotokoll ansehen, nämlich WebSocket. Im Rahmen dieses Seminars werden wir deswegen bei HTTP bleiben und lernen anstelle von Playwright das trotz der Konkurrenz immer noch etablierte Selenium kennen.

Aber was ist denn jetzt Selenium?

> Selenium automates browsers. That's it! What you do with that power is entirely up to you. Primarily it is for automating web applications for testing purposes, but is certainly not limited to just that.

Quelle: [Selenium 2023](https://www.selenium.dev/)

Selenium ist ein Projekt, das verschiedene Bibliotheken in verschiedenen Programmiersprachen umfasst und ursprünglich zum Testen von Webseiten entwickelt wurde. **Ganz allgemein ermöglich es Selenium, Webbrowser automatisiert zu steuern und mit ihnen zu interagieren. Das heißt, dass wir im Rahmen von Web Scraping mithilfe von Selenium auf die gerenderte Version einer Webseite zugreifen und direkt aus unserem Python-Notebook heraus verschiedene Aktionen im Webbrowser ausführen können, z.B. das Öffnen von URLs, Suchen nach und Klicken auf Elemente, Ausfüllen von Formularen und Extrahieren von Daten.** Um Selenium verwenden zu können, brauchen wir zusätzlich einen Browser und einen Treiber für diesen Browser (=Webdriver). In unserem Beispiel heute werden wir Selenium im Zusammenhang mit Chrome und dem [Chrome WebDriver](https://sites.google.com/chromium.org/driver/) verwenden. Der Chrome WebDriver stellt die Verbindung zum Chrome-Browser her und ermöglicht die Kommunikation zwischen Selenium und dem Browser.


### Beispiel: Zitate mit Python Selenium scrapen

Als Beispiel sehen wir uns zunächst wieder den Code von jemand anderem an, nämlich von [Lewis Kori](https://lewiskori.com/blog/beginner-s-guide-to-web-scraping-with-python-s-selenium/).

Wir nehmen allerdings zwei Änderungen vor: Da der Code für eine ältere Selenium-Version (Version 3) geschrieben wurde und sich die Funktionsnamen seither geändert haben, habe ich ein paar der Funktionsaufrufe angepasst. Außerdem verwenden wir den Präfix https anstelle von http für die quotes.toscrape Website, um eine verschlüsselte Verbindung zu der Seite herzustellen.

Zunächst müssen wir aber Selenium installieren:

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} conda-forge::selenium==4.23.1

In [ ]:
!conda list

Falls es bei der Installation ein Problem geben sollte, kann Selenium alternativ auch mithilfe des Paketmanagers pip installiert werden:

In [ ]:
!pip install selenium==4.23.1

In [ ]:
!pip list

Zusätzlich zu Selenium benötigen wir den ChromeDriver. Es gibt zwei Möglichkeiten, den ChromeDriver zu installieren. Eine gängige, aber umständliche Methode ist der manuelle Download über https://sites.google.com/chromium.org/driver/ bzw. https://googlechromelabs.github.io/chrome-for-testing/ (hier unbedingt "stable" auswählen!). Nach dem Download solltet ihr den Pfad zur heruntergeladenen Chromedriver-Datei in die Zwischenablage kopieren und in den folgenden Code einsetzen. Mit `Service(chromedriver_path)` wird dem Selenium WebDriver mitgeteilt, wo sich die ausführbare Datei des Chrome WebDriver befindet, die für die Automatisierung des Chrome-Browsers benötigt wird. Wenn webdriver.Chrome() ohne spezifische Angabe eines Service-Objekts aufgerufen wird, sucht Selenium standardmäßig im Systempfad nach dem ChromeDriver. Obwohl es in diversen Online-Foren vorgeschlagen wird, ist es nicht notwendig, den ChromeDriver dem Systempfad hinzuzufügen. Mit der Zeile `driver = webdriver.Chrome()` weisen wir Selenium an, den Chrome-Browser für die Browserautomatisierung zu verwenden und starten eine neue Session (Sitzung).


In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

chromedriver_path = "/Pfad/zum/chromedriver"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

Bei diesem Vorgehen muss jedoch beachtet werden, dass der Chromedriver immer dieselbe Version haben muss wie die Version des Chrome Browsers, den der Driver automatisiert. Wenn Chrome aktualisiert wird, muss also auch der Chromedriver aktualisiert werden. Da sich Chrome aber mitunter auch automatisch im Hintergrund aktualisiert, passiert es schnell, dass eine veraltete Chromedriver-Version verwendet wird. Dies führt dann dazu, dass Code zu einem späteren Zeitpunkt nicht mehr reproduzierbar ist.

Es gibt deswegen eine weitere und weitaus bessere Möglichkeit, den ChromeDriver zu installieren. Seit der Version 4.6 wird mit der Installation von Selenium auch der [SeleniumManager](https://www.selenium.dev/documentation/selenium_manager/) installiert, ein Modul zur automatischen Verwaltung verschiedener Chromedriver-Versionen. Dadurch müssen wir beim Aufruf von `webdriver.Chrome()` den Pfad zum installierten ChromeDriver nicht manuell angeben, sondern es wird automatisch der korrekte Webdriver heruntergeladen und der Pfad wird automatisch eingesetzt.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome() # alternativ webdriver.Firefox()

Zuletzt importieren wir noch das Paket pandas zur Verarbeitung der extrahierten Daten.

In [ ]:
import pandas as pd

In [ ]:
# Quelle: https://lewiskori.com/blog/beginner-s-guide-to-web-scraping-with-python-s-selenium/

pages = 11

total = []
for page in range(1, pages):

    url = "https://quotes.toscrape.com/js/page/" + str(page) + "/"

    driver.get(url)

    quotes = driver.find_elements(By.CLASS_NAME, "quote")
    for quote in quotes:
        quote_text = quote.find_element(By.CLASS_NAME, 'text').text[1:-2]
        author = quote.find_element(By.CLASS_NAME, 'author').text
        new = ((quote_text,author))
        total.append(new)

driver.quit()
df = pd.DataFrame(total,columns=['quote','author'])
df.to_csv('quoted.csv')

Zur Übung sollt ihr euch im Rest der Stunde selbst erarbeiten, was dieser Code macht. Schaut dazu in die Selenium-Dokumentationsseiten: https://www.selenium.dev/documentation/webdriver/elements/locators/. Bei den Codebeispielen muss jeweils Python ausgewählt werden. Etwas älter, aber manchmal immer noch hilfreich ist außerdem diese inoffizielle Dokumentationsseite: https://selenium-python.readthedocs.io/. Kommentiert jede Zeile, damit ihr euch später noch an die Ergebnisse eurer Recherche erinnern könnt: Diese Vorarbeit braucht ihr für die Übungsaufgabe!

### Quellen

```{bibliography}
   :list: enumerated
   :filter: keywords % "selenium" or keywords % "sel_1_2"
```